# Instalamos e importamos librerías

In [1]:
from builtins import round as roundpy
from builtins import max as maxpy

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f2fbe4463571bc2a4d3f7ebc1e439c9f449f06c39479e35b3d2e932a9d66b8c3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,342 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.tea

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos archivo con los datos del índice BigMac

In [5]:
#https://drive.google.com/file/d/1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG/view?usp=sharing GooglePlayStore.csv
#https://drive.google.com/file/d/1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg/view?usp=sharing GooglePlayStore_User_Reviews.csv

id1='1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG'
id2='1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg'
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('GooglePlayStore.csv')
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('GooglePlayStore_User_Reviews.csv')

# Creamos el Spark Context

In [6]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Leemos CSV

In [7]:
sqlContext = SQLContext(sc)
dfAppDetails = sqlContext.read.csv('GooglePlayStore.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
dfAppReviews = sqlContext.read.csv('GooglePlayStore_User_Reviews.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
rddAppDetails = dfAppDetails.rdd
rddAppReviews = dfAppReviews.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Preparación de Datasets

Droppeamos los apps completamente idénticos.

In [8]:
rddAppDetails = rddAppDetails.distinct()

In [9]:
rddAppDetails.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating='4.1', Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating='3.9', Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating='4.7', Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating='4.5', Reviews='215644', S

In [10]:
rddAppReviews.take(5)

[Row(App='10 Best Foods for You', Translated_Review='This help eating healthy exercise regular basis', Sentiment='Positive', Sentiment_Polarity='0.25', Sentiment_Subjectivity='0.28846153846153844'),
 Row(App='10 Best Foods for You', Translated_Review='nan', Sentiment='nan', Sentiment_Polarity='nan', Sentiment_Subjectivity='nan'),
 Row(App='10 Best Foods for You', Translated_Review='Works great especially going grocery store', Sentiment='Positive', Sentiment_Polarity='0.4', Sentiment_Subjectivity='0.875'),
 Row(App='10 Best Foods for You', Translated_Review='Best idea us', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3'),
 Row(App='10 Best Foods for You', Translated_Review='Best way', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3')]

Descartamos los reviews sin Sentiment.

In [11]:
rddAppReviews = rddAppReviews.filter(lambda x: x.Sentiment != 'nan')

# Ejercicio 10
Cuál es la categoría con mayor promedio de apps que hayan sido al menos 1 vez calificadas como positivas?

Primero filtramos los reviews positivos, y los mappeamos dejando sólo el nombre de la app del review. Luego utilizamos distinct para obtener las aplicaciones sin repetidas.

In [12]:
appsWithPositiveReviews = rddAppReviews.filter(lambda x: x.Sentiment == 'Positive').map(lambda x: x.App).distinct()
appsWithPositiveReviews.take(5)

['104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室',
 '21-Day Meditation Experience',
 '2Date Dating App, Love and matching',
 '30 Day Fitness Challenge - Workout at Home',
 '365Scores - Live Scores']

Mi idea es filtrar por las aplicaciones obtenidas en el rdd appDetails, para eso primero obtengo mediante collect dichas aplicaciones en forma de lista así utilizarla en filter.

In [13]:
appsWithPositiveReviewsList = appsWithPositiveReviews.collect()

Realizo el filter mencionado en el último texto, y también me aseguro que tengan un valor en Rating, que vamos a utilizar para calcular el promedio de Rating.

In [14]:
appsWithPositiveReviewsDetails = rddAppDetails.filter(lambda x: x.App in appsWithPositiveReviewsList and x.Rating != 'NaN')

Mappeo los detalles de las aplicaciones a (Categoría, (Rating formatteado, 1)). A la hora de formatear Rating se le eliminó el punto dado que tiene formato x.x y se lo convirtió en enteros para mejorar el perfomance y reducir errores inherentes.

In [15]:
ratingMeanPerCategory = appsWithPositiveReviewsDetails.map(lambda x: (x.Category, (int(x.Rating.replace('.','')), 1)))

Utilizo reduceByKey para conseguir (Categoría, (Suma total de Rating, Cantidad de apps)) para luego mediante map calcular el promedio de cada categoría. En este último proceso formateo esos promedios dividiéndolo por 10 dado que le habíamos sacado el punto. Mediante collect(lo utilizo sabiendo que la cantidad de categorías es acotada) observo los resultados. Como luego voy a hacer un reduce además de hacer este collect, lo cacheo.

In [16]:
ratingMeanPerCategory = ratingMeanPerCategory.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
ratingMeanPerCategory = ratingMeanPerCategory.map(lambda x: (x[0],roundpy(x[1][0]/x[1][1]/10, 3))).cache()
ratingMeanPerCategory.collect()

[('ART_AND_DESIGN', 4.3),
 ('AUTO_AND_VEHICLES', 4.575),
 ('BEAUTY', 4.283),
 ('BOOKS_AND_REFERENCE', 4.344),
 ('BUSINESS', 4.271),
 ('COMICS', 4.367),
 ('COMMUNICATION', 4.261),
 ('DATING', 4.127),
 ('EDUCATION', 4.272),
 ('ENTERTAINMENT', 4.046),
 ('EVENTS', 4.32),
 ('FINANCE', 4.287),
 ('FOOD_AND_DRINK', 4.286),
 ('HEALTH_AND_FITNESS', 4.436),
 ('HOUSE_AND_HOME', 4.153),
 ('LIBRARIES_AND_DEMO', 4.044),
 ('LIFESTYLE', 4.176),
 ('GAME', 4.436),
 ('FAMILY', 4.348),
 ('MEDICAL', 4.167),
 ('SOCIAL', 4.215),
 ('SHOPPING', 4.303),
 ('PHOTOGRAPHY', 4.334),
 ('SPORTS', 4.342),
 ('TRAVEL_AND_LOCAL', 4.247),
 ('TOOLS', 4.311),
 ('PERSONALIZATION', 4.354),
 ('PRODUCTIVITY', 4.357),
 ('PARENTING', 4.3),
 ('WEATHER', 4.3),
 ('VIDEO_PLAYERS', 4.2),
 ('NEWS_AND_MAGAZINES', 4.236),
 ('MAPS_AND_NAVIGATION', 4.386)]

La categoría con mayor promedio de apps que hayan sido al menos 1 vez calificadas como positivas es Auto And Vehicles, con 4.575 de rating promedio:

In [17]:
#ratingMeanPerCategory.max(lambda x: x[1])
#ratingMeanPerCategory.reduce(lambda a,b: a if a[1] > b[1] else b)
ratingMeanPerCategory.reduce(lambda a,b: maxpy(a,b, key=lambda x: x[1]))

('AUTO_AND_VEHICLES', 4.575)